# Session 10: Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

## 🤝 Breakout Room #1
  - Task 1: Installing Required Libraries
  - Task 2: Set Environment Variables
  - Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  - Task 4: Construct our RAG application
  - Task 5: Evaluating our Application with Ragas
  - Task 6: Making Adjustments and Re-Evaluating
  - ***Activity #1: Implement a Different Reranking Strategy***


## Task 1: Installing Required Libraries

If you have not already done so, install the required libraries using the uv package manager:
``` bash

uv sync

```


## Task 2: Set Environment Variables:

We'll also need to provide our API keys.
> NOTE: In addition to OpenAI's models, this notebook will be using Cohere's Reranker - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

You have two options for supplying your API keys in this session:
- Use environment variables (see Prerequisite #2 in the README.md)
- Provide them via a prompt when the notebook runs

The following code will load all of the environment variables in your `.env`. Then, it checks for the two API keys we need. If they are not there, it will prompt you to provide them.

First, OpenAI's for our LLM/embedding model combination!

Second, Cohere's for our reranking


In [1]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

## Task 3: Synthetic Dataset Generation for Evaluation using Ragas

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data using the Health & Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, and stress management.

Next, let's load our data into a familiar LangChain format using the `TextLoader`.

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [3]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

c:\Users\dznidaric\Documents\github\AIE9\10_Evaluating_RAG_With_Ragas\.venv\Lib\site-packages\pysbd\segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
c:\Users\dznidaric\Documents\github\AIE9\10_Evaluating_RAG_With_Ragas\.venv\Lib\site-packages\pysbd\lang\arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
c:\Users\dznidaric\Documents\github\AIE9\10_Evaluating_RAG_With_Ragas\.venv\Lib\site-packages\pysbd\lang\persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


In [4]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/4 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/9 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [5]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"Az a busy wellness-focussed professonal, how d...",[The Personal Wellness Guide A Comprehensive R...,To perform Partial Crunches for lower back pai...,single_hop_specifc_query_synthesizer
1,"How do I do partial crunches for my back pain,...",[The Personal Wellness Guide A Comprehensive R...,Partial crunches you lie on your back with kne...,single_hop_specifc_query_synthesizer
2,Howw can chamomille tee bee used too improove ...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,Herbal teas such as chamomile are mentioned as...,single_hop_specifc_query_synthesizer
3,What key information does Chapter 7 provide ab...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,Chapter 7 explains that sleep is crucial for p...,single_hop_specifc_query_synthesizer
4,what PART 7 talk about?,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,"PART 7 talk about lifestyle and wellness, incl...",single_hop_specifc_query_synthesizer
5,"According to Chapter 17, what practical strate...",[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 17 recommends several strategies to su...,single_hop_specifc_query_synthesizer
6,How does the information in Chapter 9 about un...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,"Chapter 9 discusses insomnia, highlighting its...",multi_hop_specific_query_synthesizer
7,How do the strategies for managing insomnia de...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,Chapter 9 discusses understanding and managing...,multi_hop_specific_query_synthesizer
8,How chapter 8 say to improve sleep quality and...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,Chapter 8 say to improve sleep quality you sho...,multi_hop_specific_query_synthesizer
9,How does the science of sleep discussed in Cha...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,Chapter 7 explains that sleep is essential for...,multi_hop_specific_query_synthesizer


## Task 4: Construct our RAG application

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [9]:
loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

447

### ❓ Question #1:

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

##### Answer:

Chunk overlap says how many characters will it overlap while splitting the document. The first chunk will get first 50 characters, but the second one will get 'n' characters from the first chunk. 
In this example it is 0, but let's say it is 10. First chunk will be the same, the second one would be from 40th character till 90th, and so forth. 

Its purpose is to preserve context continuity, sometimes importnat information can be lost at boundaries, or could go into "wrong" chunk, where retriever might not focus on that part or LLM could conclude it is smaller, unsignificant part of chunk.

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [8]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [9]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [10]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

Now we can produce a node for retrieval!

In [11]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### A - Augmented

Let's create a simple RAG prompt!

In [12]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### G - Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-nano` to avoid using the same model as our judge model.

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

Then we can create a `generate` node!

In [14]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [15]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [16]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [17]:
response = graph.invoke({"question" : "What exercises help with lower back pain?"})

In [18]:
response["response"]

'The provided context does not specify any exercises that help with lower back pain.'

## Task 5: Evaluating our Application with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [19]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [22]:
dataset.samples[2].eval_sample.response

'Herbal teas such as chamomile can be used to improve sleep quality for a busy professional seeking better rest by incorporating a cup of chamomile tea into their bedtime routine. Drinking chamomile tea may promote relaxation and help calm the mind, making it easier to fall asleep and enhance overall sleep quality.'

Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [23]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [24]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

Next up - we simply evaluate on our desired metrics!

In [26]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

baseline_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
baseline_result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

{'context_recall': 0.1136, 'faithfulness': 0.4182, 'factual_correctness': 0.3664, 'answer_relevancy': 0.3421, 'context_entity_recall': 0.1821, 'noise_sensitivity_relevant': 0.2934}

## Task 6: Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!




We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data_new_chunks",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data_new_chunks",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=split_documents)

adjusted_example_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [30]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_example_retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [ ]:
class AdjustedState(TypedDict):
  question: str
  context: List[Document]
  response: str

hybrid_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_adjusted, generate])
hybrid_graph_builder.add_edge(START, "retrieve_adjusted")
hybrid_graph = hybrid_graph_builder.compile()

In [ ]:
response = hybrid_graph.invoke({"question" : "How can I improve my sleep quality?"})
response["response"]

'To improve your sleep quality, you can adopt several sleep hygiene practices such as maintaining a consistent sleep schedule, creating a relaxing bedtime routine, and keeping your bedroom cool, dark, and quiet. Limiting screen exposure 1-2 hours before bed, avoiding caffeine after 2 PM, and exercising regularly but not too close to bedtime can also help. Additionally, using blackout curtains or a sleep mask, ensuring your mattress and pillows are comfortable, and following a daily wellness checklist—including getting 7-9 hours of sleep and practicing mindfulness—can further enhance sleep quality. Natural remedies like relaxation techniques, herbal teas, or consulting a healthcare provider about magnesium supplements and CBT-I may also support better sleep.'

In [ ]:
import time
import copy

hybrid_dataset = copy.deepcopy(dataset)

# 10 calls per 60 seconds = 6 seconds between calls
CALLS_PER_MINUTE = 10
SECONDS_BETWEEN_CALLS = 60 / CALLS_PER_MINUTE  # 6 seconds

for idx, test_row in enumerate(hybrid_dataset):
    start_time = time.time()
    
    response = hybrid_graph.invoke({"question": test_row.eval_sample.user_input})
    test_row.eval_sample.response = response["response"]
    test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
    
    # Calculate elapsed time and wait if needed
    elapsed = time.time() - start_time
    wait_time = SECONDS_BETWEEN_CALLS - elapsed
    
    if wait_time > 0 and idx < len(hybrid_dataset) - 1:  # Don't wait after last call
        print(f"Waiting {wait_time:.2f}s to respect rate limit...")
        time.sleep(wait_time)


Waiting 3.32s to respect rate limit...
Waiting 3.20s to respect rate limit...
Waiting 3.31s to respect rate limit...
Waiting 2.74s to respect rate limit...
Waiting 3.96s to respect rate limit...
Waiting 3.14s to respect rate limit...
Waiting 0.94s to respect rate limit...
Waiting 0.13s to respect rate limit...
Waiting 1.70s to respect rate limit...
Waiting 1.83s to respect rate limit...


In [ ]:
hybrid_dataset.samples[0].eval_sample.response

'To perform Partial Crunches correctly and help with lower back pain, lie on your back with your knees bent and cross your arms over your chest. Tighten your stomach muscles and raise your shoulders off the floor, keeping your neck relaxed. Hold briefly at the top, then lower your shoulders back down with control. Ensure you are engaging your core muscles throughout the movement and avoid pulling on your neck or head. \n\nThe recommended number of repetitions for Partial Crunches is 8-12.'

In [ ]:
rerank_evaluation_dataset = EvaluationDataset.from_pandas(hybrid_dataset.to_pandas())

In [41]:
rerank_evaluation_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'reference_contexts', 'response', 'reference'], len=11)

In [40]:
rerank_result = evaluate(
    dataset=rerank_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
rerank_result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

{'context_recall': 0.6654, 'faithfulness': 0.5922, 'factual_correctness': 0.6555, 'answer_relevancy': 0.9463, 'context_entity_recall': 0.3639, 'noise_sensitivity_relevant': 0.1597}

### ❓ Question #2:

Which system performed better, on what metrics, and why?

##### Answer:
Baseline result: {'context_recall': 0.1136, 'faithfulness': 0.4182, 'factual_correctness': 0.3664, 'answer_relevancy': 0.3421, 'context_entity_recall': 0.1821, 'noise_sensitivity_relevant': 0.2934}


Rerank result: {'context_recall': 0.6654, 'faithfulness': 0.5922, 'factual_correctness': 0.6555, 'answer_relevancy': 0.9463, 'context_entity_recall': 0.3639, 'noise_sensitivity_relevant': 0.1597}

Rerank system performed better on all metrics.

Context recall - reranking surfaces more relevant documents that contain the information needed to answer questions, dramatically improving retrieval quality.

Faithfulness - better retrieved context means the LLM has more accurate source material to ground its responses in, reducing hallucinations.​

Factual correctness - higher context quality provides the LLM with correct facts, leading to more factually accurate answers.​

Answer relevancy - reranking filters out irrelevant documents before generation, allowing the LLM to produce focused, on-topic responses.

Context entity recall - more precise retrieval captures more key entities from the documents that are necessary for complete answers.​

Noise sensitivity - relevant mode measures errors when using relevant contexts, better document ranking reduces exposure to irrelevant information, making the system more robust and less prone to misinterpretation.


### ❓ Question #3:

What are the benefits and limitations of using synthetic data generation for RAG evaluation? Consider both the practical advantages and potential pitfalls.

##### Answer:
Synthetic data generation for RAG evaluation offers significant practical advantages: it's fast, cost-effective compared to manual annotation, and enables controlled testing of edge cases and complex multi-hop scenarios. However, it has critical limitations, synthetic queries often lack real world examples and consistently underestimate task difficulty. Even though we create/order LLM to provide edge cases, it cann't know all. The quality depends entirely on source documents and generator capabilities, and using the same LLM family for both generation and evaluation can create misleadingly optimistic results. 

Best practice (like always is "hybrid") is combining synthetic data with real user queries and implementing human review rather than relying solely on synthetic evaluation.


### ❓ Question #4:

If you were building a production wellness assistant, which Ragas metrics would be most important to optimize for and why? Consider the healthcare/wellness domain specifically.

##### Answer:
Most important would be faithfulness, factual correctness and context recall; then we would consider other metrics. 
Healthcare responses must be grounded in retrieved medical content to avoid hallucinations that could mislead users about health conditions or treatments, since medical misinformation can directly harm patients. ChatGpt and perhaps other llm providers state that they cann't give personalized medical advice: "I cannot diagnose conditions, prescribe treatments, or tailor advice specifically to you — only a licensed healthcare professional can do that." The system must retrieve all relevant medical information, and all of that information must be evidence-based; incorrect facts about symptoms, medications, or treatments pose serious safety risks. 

## Activity #1: Implement a Different Reranking Strategy

In this activity, you'll experiment with different reranking parameters or strategies to see how they affect the evaluation metrics.

**Requirements:**
1. Modify the `retrieve_adjusted` function to use different parameters (e.g., change `k` values, try different top_n for reranking)
2. Or implement a different retrieval enhancement strategy (e.g., hybrid search, query expansion)
3. Run the evaluation and compare results with the baseline and reranking results above
4. Document your findings in the markdown cell below

Hybrid Search (Vector + Keyword)

In [58]:
!uv pip install rank_bm25

Resolved 2 packages in 325ms
Prepared 1 package in 144ms
Installed 1 package in 10ms
 + rank-bm25==0.2.2


In [59]:
### YOUR CODE HERE ###

# Implement your custom retrieval strategy here
# Example: modify retrieve_adjusted with different parameters

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

def retrieve_hybrid(state):
    """
    Combines vector similarity with BM25 keyword matching.
    Hypothesis: Hybrid captures both semantic and exact-match queries.
    """
    
    bm25_retriever = BM25Retriever.from_documents(
        documents=split_documents,  # Replace with your doc collection
        k=5
    )
    
    # Combine vector and BM25 with equal weights
    ensemble_retriever = EnsembleRetriever(
        retrievers=[adjusted_example_retriever, bm25_retriever],
        weights=[0.5, 0.5]
    )
    
    # Apply reranking on top of hybrid results
    compressor = CohereRerank(model="rerank-v3.5", top_n=4)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=ensemble_retriever
    )
    
    retrieved_docs = compression_retriever.invoke(state["question"])
    return {"context": retrieved_docs}


In [60]:
class AdjustedState(TypedDict):
  question: str
  context: List[Document]
  response: str

hybrid_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_hybrid, generate])
hybrid_graph_builder.add_edge(START, "retrieve_hybrid")
hybrid_graph = hybrid_graph_builder.compile()

In [62]:
import time
import copy

hybrid_dataset = copy.deepcopy(dataset)

# 10 calls per 60 seconds = 6 seconds between calls
CALLS_PER_MINUTE = 10
SECONDS_BETWEEN_CALLS = 61 / CALLS_PER_MINUTE  # 6 seconds

for idx, test_row in enumerate(hybrid_dataset):
    start_time = time.time()
    
    response = hybrid_graph.invoke({"question": test_row.eval_sample.user_input})
    test_row.eval_sample.response = response["response"]
    test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
    
    # Calculate elapsed time and wait if needed
    elapsed = time.time() - start_time
    wait_time = SECONDS_BETWEEN_CALLS - elapsed
    
    if wait_time > 0 and idx < len(hybrid_dataset) - 1:  # Don't wait after last call
        print(f"Waiting {wait_time:.2f}s to respect rate limit...")
        time.sleep(wait_time)


Waiting 2.57s to respect rate limit...
Waiting 3.23s to respect rate limit...
Waiting 3.49s to respect rate limit...
Waiting 2.72s to respect rate limit...
Waiting 3.31s to respect rate limit...
Waiting 2.88s to respect rate limit...
Waiting 0.27s to respect rate limit...
Waiting 1.36s to respect rate limit...
Waiting 2.11s to respect rate limit...
Waiting 2.59s to respect rate limit...


In [63]:
hybrid_dataset.samples[0].eval_sample.response

'As a busy wellness-focused professional, to perform Partial Crunches correctly and help with lower back pain, follow these steps based on the provided guidance:\n\n1. Lie on your back with your knees bent.\n2. Cross your arms over your chest.\n3. Tighten your stomach muscles (engage your core).\n4. Carefully raise your shoulders off the floor, avoiding pulling on your neck or head.\n5. Hold the lifted position briefly.\n6. Lower your shoulders back down slowly and controlled.\n\nThe recommended number of repetitions for Partial Crunches is 8 to 12. Performing this exercise within this range can help strengthen your core, which may alleviateg or prevent lower back pain.'

In [65]:
from ragas import EvaluationDataset

evaluation_hybrid_dataset = EvaluationDataset.from_pandas(hybrid_dataset.to_pandas())

In [66]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

In [67]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

hybrid_result = evaluate(
    dataset=evaluation_hybrid_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
hybrid_result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

{'context_recall': 0.7149, 'faithfulness': 0.6419, 'factual_correctness': 0.6609, 'answer_relevancy': 0.9425, 'context_entity_recall': 0.4055, 'noise_sensitivity_relevant': 0.1054}

### Activity #1 Findings:

*Document your findings here: What strategy did you try? How did it compare to the baseline and reranking results?*

**Baseline result**: {'context_recall': 0.1136, 'faithfulness': 0.4182, 'factual_correctness': 0.3664, 'answer_relevancy': 0.3421, 'context_entity_recall': 0.1821, 'noise_sensitivity_relevant': 0.2934}

**Rerank result**: {'context_recall': 0.6654, 'faithfulness': 0.5922, 'factual_correctness': 0.6555, 'answer_relevancy': 0.9463, 'context_entity_recall': 0.3639, 'noise_sensitivity_relevant': 0.1597}

**Hybrid result**: {'context_recall': 0.7149, 'faithfulness': 0.6419, 'factual_correctness': 0.6609, 'answer_relevancy': 0.9425, 'context_entity_recall': 0.4055, 'noise_sensitivity_relevant': 0.1054}


Hybrid search outperformed both baseline and reranker. The combination of semantic understanding (vector search) and exact-match capabilities (BM25) provides superior recall and entity recognition critical for healthcare where missing specific terms could be dangerous. The improved faithfulness and noise sensitivity make it the most reliable strategy.
As we said before 3 metrics are most important for our welness rag app: faithfulness, factual correctness and context recall; which here are better, but I think still not ready for production deployment.